In [8]:
import numpy as np

import tsplib95
import networkx as nx
from networkx.algorithms.approximation import christofides

import os
import pandas as pd
import gilsrvnd
import DBMEA
import grasp




In [51]:
# read in graph instances/random_20_0.tsp
problem = tsplib95.load('Instances/att48.tsp')
graph = problem.get_graph()
dist_matrix = nx.to_numpy_matrix(graph)



In [52]:
t = christofides(graph)
# remove last index of t
t = [x - 1 for x in t]
t.pop()
t = list(t)
t

[0,
 7,
 21,
 15,
 2,
 40,
 33,
 13,
 24,
 38,
 47,
 4,
 41,
 9,
 34,
 44,
 23,
 25,
 3,
 1,
 28,
 31,
 20,
 46,
 10,
 12,
 22,
 11,
 14,
 39,
 8,
 32,
 19,
 45,
 17,
 35,
 5,
 29,
 42,
 26,
 16,
 18,
 36,
 27,
 6,
 43,
 30,
 37]

In [55]:
def cost(s, dist_matrix):
    cost = sum((len(s) - i - 1) * dist_matrix[s[i], s[i + 1]] for i in range(len(s) - 1))
    return cost

In [56]:
cost(t, dist_matrix)

333391.0

In [34]:
# read in matrix_all csv
matrix_all = pd.read_csv('matrix_good.csv')
matrix_all

,file,n,m,ratio1,ratio2,density,min,max,mean,std,var,GILS_cost,GILS_time,GRASP_cost,GRASP_time,DBMEA_cost,DBMEA_time,Min_val,Min_col,min_method
0,att48.tsp,48,1176,0.040816,24.5,1.042553,0.0,2662.0,1017.559896,627.598414,3.938798e+05,226229.0,6.474685,301702.0,0.026019,207771.0,5.288543,0,NaN,DBMEA_cost
1,berlin52.tsp,52,1378,0.037736,26.5,1.039216,0.0,1716.0,564.188609,345.722052,1.195237e+05,145419.0,6.054587,191072.0,0.031533,143278.0,5.374160,0,NaN,DBMEA_cost
2,brazil58.tsp,58,1711,0.033898,29.5,1.035088,0.0,8700.0,2094.914388,1426.139588,2.033874e+06,532454.0,8.013889,709540.0,0.043413,535150.0,9.045840,0,NaN,GILS_cost
3,burma14.tsp,14,105,0.133333,7.5,1.153846,1.0,1261.0,442.612245,272.511473,7.426250e+04,16457.0,0.156218,18393.0,0.001000,16457.0,0.082072,0,NaN,GRASP_time
4,dantzig42.tsp,42,903,0.046512,21.5,1.048780,0.0,192.0,72.295918,43.481585,1.890648e+03,12392.0,2.579336,14523.0,0.016044,12256.0,3.325098,0,NaN,DBMEA_cost
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,line_90_5.tsp,90,4095,0.021978,45.5,1.022472,0.0,889.0,319.538272,226.908988,5.148769e+04,55730.0,35.477098,157350.0,0.233857,57558.0,36.326975,0,NaN,GILS_cost
336,line_90_6.tsp,90,4095,0.021978,45.5,1.022472,0.0,888.0,297.687407,211.917802,4.490915e+04,48884.0,30.191444,128976.0,0.232372,45916.0,37.348990,0,NaN,DBMEA_cost
337,line_90_7.tsp,90,4095,0.021978,45.5,1.022472,0.0,862.0,249.932840,181.712429,3.301941e+04,51464.0,28.821121,124368.0,0.232510,49038.0,44.483338,0,NaN,DBMEA_cost
338,line_90_8.tsp,90,4095,0.021978,45.5,1.022472,0.0,889.0,316.810123,226.894830,5.148126e+04,60845.0,28.282006,123373.0,0.231058,65825.0,39.745050,0,NaN,GILS_cost


In [29]:
matrix_all["Min_col"] = ""
matrix_all["Min_val"] = 0

In [41]:
# find the minimum of the columns GILS_cost, DBMEA_cost, GRASP_cost, in case of a tie choose the column with the minimum time column
def findBest(row):
    cost_columns = ['GRASP_cost', 'GILS_cost', 'DBMEA_cost']
    min_cost = row[cost_columns].min()

    min_columns = row[[col for col in cost_columns if row[col] == min_cost]]

    if len(min_columns) == 1:
        return min_columns.index[0]
    else:
        time_columns = ["GRASP_time", "GILS_time", "DBMEA_time"]
        min_time = row[time_columns].min()
        min_time_columns = row[[col for col in time_columns if row[col] == min_time]]
        return min_time_columns.index[0]

In [42]:
matrix_all['min_method'] = matrix_all.apply(findBest, axis=1)

In [44]:
matrix_all["min_method"].value_counts()

min_method
DBMEA_cost    186
GILS_cost     121
GRASP_time     33
Name: count, dtype: int64

In [45]:
# save matrix_all as csv
#matrix_all.to_csv('matrix_good.csv', index=False)